# Project Presentation: ROS basics 

The purpose of this report is to demonstate, both in simulation and in a real robot, the use of ROS platform to navigate a robot in a predefined enviroment which consists of four walls. The robot, in general, has to move along the walls. Apart from the navigation itself, a service and an action node are developed.  

## Package information

**Name:** robot_control_main



### Running in Simulation mode
To run the simulation the following commands are necessary. 

**Shell #1**
```
roslaunch realrobotlab main.launch
```

**Shell #2**
```
roslaunch robot_control_main main.launch
```

(Optional) It will be helpful to use the teleop control for initially positioning the robot

**Shell #3**
```
roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch
```

**Make sure to interrupt the remote control while running the main node!!!**

### Running in RobotLab
As soon as the enviroment is loaded, use only the same command to launch the program, and/or the teleop control. 

## Robot behaviour
The robot itself is a **Turtebot3** one and a screenshot of the enviroment is shown below:

<img src="new_sim.png"/>


The robot should apply the following **general behaviour**, in chronological order:
- The robot from its starting position should move to the closest wall
- The robot will navigate in parallel to the wall
- When the robot completes one lap, it should stop and return the list of odometries recorded

In more detail, there are some further requirements and specifications that are explained analytically in the specific sections.


## Approach to the solution
The code structure consists of 8 python files. 3 are used for interacting with the topics, 1 for the findwall service server, 1 for the odometry action server and 1 is the main behaviour node. A config file defines all the parameters and an extra postprocessing is proving a graph of the orbit. 

### Topics
There are three topics in total required for the control of the robot and the feedback from the enviroment:
- **/cmd_vel**: A publisher node that controls the movement of the robot. Values are assigned to the *linear.x* and *angular.z* parts  
- **/scan**: A subscriber to the laser scanning of the enviroment, showing distance to obstacles in a 360 size array. Front is at index 180 and left and right at 90 and 270 respectively. 
- **/odom**: A subscriber that shows the position of the robot. It is only used in the action that measure the covered distance. 

The following three files define objects and functions to the mentioned topics:
- **cmd_vel_publisher.py**: Defines the object **CmdVelPub** with main fuction:
    - move_robot( direction, linearspeed=0.0, angularspeed=0.0): Moves robot to a direction with specific speed values
- **laser_subscriber.py**: Defines the object **LaserTopicReader** with main fuction:
  - wall_detector ( ): Returns three reading values, for the front, left and right directions
- **odom_subscriber.py**: Defines the object **OdomTopicReader** with main fuction:
  - wall_detector ( ): Returns three reading values, for the front, left and right directions

### Nodes
Since there three distinct parts of the behaviour, three nodes are defined.
- **find_wall_service_server.py**: Defines the service */find_wall* that when called positions the robot in a predefined initial position
- **record_odom_action_server.py**: Defines the */record_odom* topic as an action 
- **main_program.py**: The top level program that defines the *robot_control_main_node* and performs the specified behaviour.




![rosgraph](rosgraph.png)



### Launch
In the launch file, the service and action servers are launched and then the main program executes as shown in the code:

**main.launch**

```
<launch>
<node pkg ="robot_control_main"
    type="record_odom_action_server.py"
    name="record_odom_action_server_node"
    output="screen">
</node>

<node pkg ="robot_control_main"
    type="find_wall_service_server.py"
    name="find_wall_node"
    output="screen">
</node>

<node pkg ="robot_control_main"
    type="main_program.py"
    name="robot_control_main_node"
    output="screen">
</node>

</launch>

```


## Starting position service
The **find_wall** service takes an Empty input and returns the boolean *wallfound*. 

### Detailed behaviour for initial positioning
The robot is initially at an arbirtrary position and has to reposition itself according to the following sequence:
- Turn around to face the closest wall (Rotation #1)
- Move forward to a distance of 0.3m from the wall (Positioning)
- Turn around 90 degrees to have the wall on the right side (Rotation #2)

In the code two variables are used to check for the progress and the completion of the service, *rotated* and *positioned* and start at False.

- **Rotation 1**: Robot rurns around till the laser scan at front (index 180) has the minimum value. When done, **rotated = True**
- **Positioning**: Robot moves till laser reading of front is <= 0.3m.  When done, **positioned = True, rotated = False**
- **Rotation 2**: Robot turns 90 degrees laser scan at right has minimum value. When done, **positioned = True, rotated = True**

Then the service completes and **wallfound=True**

### Note
The robot does not have advanced intelligence therefore when an obstacle is detected, it is considered to be a wall. Since the environment can have some other objects we need to make sure they stay away from the wall in order not to interfere with the robot. 


## Navigation node
Once positioned and receiving the **wallfound** message, the robot starts moving in parallel the closest wall. It constantly reads the front and right distances with the laser sensor and adapts the angular speed to avoid wall at the front and to keep a constant distance from the right side.  

The wall following behavior is a behavior that makes the robot follow along the wall on its right hand side. This means that the robot must be moving forward at a 30cm (1 foot) distance from the wall, having the wall on its right hand side, the entire time.
- If the ray distance is bigger than 0.3m, you need to make the robot approach the wall a little, by adding some rotational speed to the robot
- If the ray distance is lower than 0.2m, you need to move the robot away from the wall, by adding rotational speed in the opposite direction
- If the ray distance is between 0.2m and 0.3m, just keep the robot moving forward

The behaviour is implemented in the main node and continues till it completes a lap, a signal that is sent by the odometry action.

    
## Odometry action

The odometry action start with the navigation node. It has no goal but it is considered complete when the robot is close to its initial position after completing a lap. To do so, when the node start the initial pose is captured and there is always a check to see how close its current position is to the initial one. 

To avoid sending a success signal early, we consider that the robot has to move at least 1m from its initial position. Additionally, to avoid missing the target after the lap, a radius of 0.25m around the initial position is considered a margin. If by any chance the robot fails to pass closer to that distance, after 10m it will stop nevertheless. 

As a feedback the robot returns the distance covered so far. At a rate of 1sec, the linear distance between two successive points is added to the total one. 

The result after the action is finished, is an array of the points the robot's orbit.

Once action is complete, the robot stops and its mission is accomplished. 


    
## Results

The code is tested both in simulation and the real robot and the results are shown in the following videos.

### Testing the package in simulation environment

[![RobotX](http://img.youtube.com/vi/ZoI2RdrBV6Y/0.jpg)](https://youtu.be/ZoI2RdrBV6Y "RobotX")

### Testing the package in Real Robot

[![RobotX](http://img.youtube.com/vi/vXqvZJl8p2o/0.jpg)](https://youtu.be/vXqvZJl8p2o "RobotX")

### Orbit from the action
To demonstrate that the action returns the correct results, the points are plotted as shown below and this behaviour is the expected one. 
![orbit](orbit.png)


## Improvements
After the initial development of the solution and observation of the behaviour there are some points of improvement. Mainly, the navigation can be a bit more elaborated from a fixed linear and angular speed. 